In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pickle
import pandas as pd
import numpy as np
from copy import copy
import tensorflow as tf
from sklearn.model_selection import train_test_split

import pymedtermino
import seaborn as sns

from metric_learn import NCA
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier

from tpot import TPOTClassifier

In [3]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

import sys
sys.path.append(f'../../')
import os
os.chdir('../../')

from src.data.sentence_vectorizer import SentenceVectorizer
from src.data.tokenizer import Tokenizer

Create random configuration
create config: 'temp/run_config_default_run_name.yml'


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kaigorodov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kaigorodov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
cadec_train = pd.read_csv('data/interim/cadec/train.csv')
cadec_test = pd.read_csv('data/interim/cadec/test.csv')
cadec_train.shape, cadec_test.shape

((3520, 8), (881, 8))

In [5]:
cadec_train

,term,start,end,sent,text,code,STR,SNMS
0,lethargy,0,8,lethargy and leg soreness.,lethargy and leg soreness.,10024264,Lethargy,"['Lethargy', 'Lethargy', 'Lethargy', 'Lethargi..."
1,Pain in the shoulders,34,55,A tingling sensation in the back of my neck.,Joint pain in the knees and hips.<SENT>Pain in...,10040617,Shoulder pain,"['Shoulder pain', 'Shoulder pain', 'Shoulder p..."
2,unable to walk,128,142,I stopped it 8 weeks ago and still have much m...,"Bouts of extreme forgetfulness, extreme muscle...",10049278,Unable to walk,"['Unable to walk (finding)', 'Unable to walk']"
3,hematuria,30,39,Lipitor caused permanent liver and kidney dama...,"Severe back pain, flank pain, hematuria, kidne...",10018867,['Hematuria' 'Haematuria'],"['Hematuria', 'Hematuria', 'Hematuria', 'Haema..."
4,Extreme vertigo,33,48,In decreasing order of severity: Extreme verti...,In decreasing order of severity: Extreme verti...,10047340,Vertigo,"['Vertigo', 'Vertigo', 'Vertigo', 'Vertigo, NO..."
...,...,...,...,...,...,...,...,...
3515,first vaginal bleeding,488,510,More problems and this time I had the ideo to ...,"I have vaginal bleeding, my cycle was back (I'...",10046883,Vaginal bleeding,"['Vaginal hemorrhage', 'Vaginal bleeding', 'Va..."
3516,significant limp,259,275,The drug worked wonders on cholesterol (down 8...,Left shoulder pain and right hip and muscle pa...,10058726,Limping,"['Lameness', 'Limping', 'Limp', 'Limping (find..."
3517,slightly blurred vision,50,73,All the symptoms largely disappeared after two...,"rapid shallow breathing, fatigue, disorientati...",10005886,Blurred vision,"['Blurred vision', 'Blurred vision', 'Blurred ..."
3518,tendonitis,46,56,No relief from any pain pills.,Severe shoulder and upper body pain including ...,10066371,Tendon pain,"['Tenalgia', 'Tendon pain', 'Tenodynia', 'Tena..."


In [6]:
cadec_train['terms'] = cadec_train.apply(lambda row: row['term'] + ', ' + row['STR'], axis=1)
cadec_train

,term,start,end,sent,text,code,STR,SNMS,terms
0,lethargy,0,8,lethargy and leg soreness.,lethargy and leg soreness.,10024264,Lethargy,"['Lethargy', 'Lethargy', 'Lethargy', 'Lethargi...","lethargy, Lethargy"
1,Pain in the shoulders,34,55,A tingling sensation in the back of my neck.,Joint pain in the knees and hips.<SENT>Pain in...,10040617,Shoulder pain,"['Shoulder pain', 'Shoulder pain', 'Shoulder p...","Pain in the shoulders, Shoulder pain"
2,unable to walk,128,142,I stopped it 8 weeks ago and still have much m...,"Bouts of extreme forgetfulness, extreme muscle...",10049278,Unable to walk,"['Unable to walk (finding)', 'Unable to walk']","unable to walk, Unable to walk"
3,hematuria,30,39,Lipitor caused permanent liver and kidney dama...,"Severe back pain, flank pain, hematuria, kidne...",10018867,['Hematuria' 'Haematuria'],"['Hematuria', 'Hematuria', 'Hematuria', 'Haema...","hematuria, ['Hematuria' 'Haematuria']"
4,Extreme vertigo,33,48,In decreasing order of severity: Extreme verti...,In decreasing order of severity: Extreme verti...,10047340,Vertigo,"['Vertigo', 'Vertigo', 'Vertigo', 'Vertigo, NO...","Extreme vertigo, Vertigo"
...,...,...,...,...,...,...,...,...,...
3515,first vaginal bleeding,488,510,More problems and this time I had the ideo to ...,"I have vaginal bleeding, my cycle was back (I'...",10046883,Vaginal bleeding,"['Vaginal hemorrhage', 'Vaginal bleeding', 'Va...","first vaginal bleeding, Vaginal bleeding"
3516,significant limp,259,275,The drug worked wonders on cholesterol (down 8...,Left shoulder pain and right hip and muscle pa...,10058726,Limping,"['Lameness', 'Limping', 'Limp', 'Limping (find...","significant limp, Limping"
3517,slightly blurred vision,50,73,All the symptoms largely disappeared after two...,"rapid shallow breathing, fatigue, disorientati...",10005886,Blurred vision,"['Blurred vision', 'Blurred vision', 'Blurred ...","slightly blurred vision, Blurred vision"
3518,tendonitis,46,56,No relief from any pain pills.,Severe shoulder and upper body pain including ...,10066371,Tendon pain,"['Tenalgia', 'Tendon pain', 'Tenodynia', 'Tena...","tendonitis, Tendon pain"


In [13]:
sentvec = SentenceVectorizer()
train = sentvec.vectorize(cadec_train, vectorizer_name='word2vec')

LOADING MODEL


  3%|▎         | 92/3520 [00:00<00:30, 112.79it/s]/home/kaigorodov/myprojects/MedConcNorm/src/data/sentence_vectorizer.py:107: RuntimeWarning: Mean of empty slice.
  return vecs.mean(axis=0)
100%|██████████| 3520/3520 [00:28<00:00, 121.39it/s]


In [14]:
train

,term,start,end,sent,text,code,STR,SNMS,terms,term_vec
0,lethargy,0,8,lethargy and leg soreness.,lethargy and leg soreness.,10024264,Lethargy,"['Lethargy', 'Lethargy', 'Lethargy', 'Lethargi...","lethargy, Lethargy","[0.21521164, -0.026344152, 0.38549376, 0.25564..."
1,Pain in the shoulders,34,55,A tingling sensation in the back of my neck.,Joint pain in the knees and hips.<SENT>Pain in...,10040617,Shoulder pain,"['Shoulder pain', 'Shoulder pain', 'Shoulder p...","Pain in the shoulders, Shoulder pain","[-0.47513, 0.048219055, 0.3915003, -0.14335209..."
2,unable to walk,128,142,I stopped it 8 weeks ago and still have much m...,"Bouts of extreme forgetfulness, extreme muscle...",10049278,Unable to walk,"['Unable to walk (finding)', 'Unable to walk']","unable to walk, Unable to walk","[-0.08633718, 0.14751735, 0.2959479, -0.011275..."
3,hematuria,30,39,Lipitor caused permanent liver and kidney dama...,"Severe back pain, flank pain, hematuria, kidne...",10018867,['Hematuria' 'Haematuria'],"['Hematuria', 'Hematuria', 'Hematuria', 'Haema...","hematuria, ['Hematuria' 'Haematuria']","[0.13927992, -0.15071428, 0.06184798, -0.36352..."
4,Extreme vertigo,33,48,In decreasing order of severity: Extreme verti...,In decreasing order of severity: Extreme verti...,10047340,Vertigo,"['Vertigo', 'Vertigo', 'Vertigo', 'Vertigo, NO...","Extreme vertigo, Vertigo","[-0.017812165, 0.06832041, -0.21009833, 0.1160..."
...,...,...,...,...,...,...,...,...,...,...
3515,first vaginal bleeding,488,510,More problems and this time I had the ideo to ...,"I have vaginal bleeding, my cycle was back (I'...",10046883,Vaginal bleeding,"['Vaginal hemorrhage', 'Vaginal bleeding', 'Va...","first vaginal bleeding, Vaginal bleeding","[0.25300187, -0.08618194, -0.18330447, -0.0156..."
3516,significant limp,259,275,The drug worked wonders on cholesterol (down 8...,Left shoulder pain and right hip and muscle pa...,10058726,Limping,"['Lameness', 'Limping', 'Limp', 'Limping (find...","significant limp, Limping","[-0.24350393, 0.030809045, 0.34279796, 0.06854..."
3517,slightly blurred vision,50,73,All the symptoms largely disappeared after two...,"rapid shallow breathing, fatigue, disorientati...",10005886,Blurred vision,"['Blurred vision', 'Blurred vision', 'Blurred ...","slightly blurred vision, Blurred vision","[-0.0014076779, 0.42433706, 0.033752162, -0.24..."
3518,tendonitis,46,56,No relief from any pain pills.,Severe shoulder and upper body pain including ...,10066371,Tendon pain,"['Tenalgia', 'Tendon pain', 'Tenodynia', 'Tena...","tendonitis, Tendon pain","[-0.62536067, -0.07171742, 0.32931018, -0.3697..."


In [74]:
cadec_test['term']

0                   light nausea
1          pain in feet and legs
2                       headache
3                        Malaise
4                  slurpy speach
                 ...            
876              Lower back pain
877                          gas
878               Pain in joints
879             perforated ulcer
880    calves muscle pain in the
Name: term, Length: 881, dtype: object

In [75]:
test = sentvec.vectorize_tfidf(cadec_test, feat_col='term')

In [76]:
test

,term,start,end,sent,text,code,STR,SNMS,term_vec
0,light nausea,126,138,"For the first 8 days of ever taking it, the on...","For the first 8 days of ever taking it, the on...",10028813,Nausea,"['Nausea', 'Nausea', 'Nausea', 'Nausea', 'Naus...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,pain in feet and legs,11,32,"Still have pain in feet and legs, since Dr mad...","Still have pain in feet and legs, since Dr mad...",10016974,Foot pain,"['Foot pain', 'Foot pain', 'Foot pain', 'Foot ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,headache,494,502,I was prescribed this drug because I had spent...,I was prescribed this drug because I had spent...,10019211,Headache,"['Headache', 'Headache', 'Headache', 'Headache...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Malaise,0,7,"Malaise, exhaustion, severe muscle pain, sever...","Malaise, exhaustion, severe muscle pain, sever...",10025482,Malaise,"['Malaise', 'Malaise', 'Malaise', 'Ill', 'Vagu...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,slurpy speach,14,27,"slurpy speach, mussle spasm in the toes and le...","blurr vision.<SENT>slurpy speach, mussle spasm...",10041053,Slurred speech,"['Slurred speech', 'Slurred speech', 'Slurring...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...
876,Lower back pain,65,80,Stopped taking Lipitor 40 ml.,"Weakness in both legs, hip pain, knee pain, fe...",10024891,Low back pain,"['Low back pain', 'Low back pain', 'Low back p...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
877,gas,409,412,"I also find that it makes me a bit drowsy, alm...",I used to take Provachol (sp?)but I found it c...,10016766,Flatulence,"['Flatulence', 'Flatulence', 'Flatulence', 'Fl...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
878,Pain in joints,8,14,"Pain in joints and muscles, including knees, a...","Pain in joints and muscles, including knees, a...",10003239,Arthralgia,"['Arthralgia', 'Arthralgia', 'Arthralgia', 'Ar...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
879,perforated ulcer,52,68,Slight stomach upset became worse and resulted...,Slight stomach upset became worse and resulted...,10000762,Acute gastric ulcer with perforation,"['Acute gastric ulcer with perforation', 'Acut...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [ ]:
X_train, y_train = train['term_vec'], train['code']
X_test, y_test = test['term_vec'], test['code']

X_train = pd.DataFrame([pd.Series(x) for x in X_train])
X_test = pd.DataFrame([pd.Series(x) for x in X_test])

In [ ]:
labels = pd.concat([y_train, y_test])
labels.shape

In [ ]:
meddra_labels = {v:k for k, v in enumerate(labels.unique())}
y_train = y_train.apply(lambda x: int(meddra_labels[x]))
y_test = y_test.apply(lambda x: int(meddra_labels[x]))

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
generations = 2
population_size = 50
max_eval_time_mins = 2
n_jobs = 10
max_iter = 10

tpot = TPOTClassifier(generations=generations, 
                      population_size=population_size,
                      verbosity=2, 
                      random_state=42, 
                      max_eval_time_mins=max_eval_time_mins, 
                      n_jobs=n_jobs)

tpot.fit(X_train, y_train)
score = tpot.score(X_test, y_test)
logging.debug(f'PURE MEDDRA: {score}')

In [ ]:
score

In [ ]:
1+1

In [ ]:
clf.score(X, y)